In [1]:
! pip install Keras

In [2]:
#warnings關掉
import warnings
warnings.filterwarnings('ignore')
#輸入datasets
from sklearn import datasets
#pandas可以提供高效能、簡易使用的資料格式(DataFrame)，讓使用者可以快速操作及分析資料
import pandas as pd
#數學公式計算都靠它
import numpy as np
#畫圖都靠它
import matplotlib.pyplot as plt
#此套件可將資料自由切分成 訓練資料集 和 測試資料集
from sklearn.model_selection import train_test_split
#標準化資料集
from sklearn.preprocessing import minmax_scale
#DNN在Keras分類器演算法的套件
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
#計算accuracy,recall,precision測量指標
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix

Using TensorFlow backend.


### 二、使用安德森鳶尾花卉數據集(Iris dataset)來做數據分析-資料前處理-雙類別
Iris 資料集的介紹：http://bit.ly/2ptEM0N （連結到wiki)

In [3]:
#輸入資料集
iris = datasets.load_iris()

In [4]:
#iris['data']是資料內容
#數據標準化(normalization)主要解決不同性質數據問題，讓每一個性質的資料可以在同一個起跑點作分析
#常見方法有:Max-Min scalar; z-score statistic scalar; maxabs scaler; robust scaler等
#此處我將使用Max-Min scalar做示範
x_iris = minmax_scale(iris['data'],feature_range=(0, 1), axis=0, copy=True)

In [5]:
#存取成dataFrame形式:  iris['feature_names']是資料標題
x_iris = pd.DataFrame(x_iris, columns=iris['feature_names'])
#只要看到".head()"都是pandas呈現資料用
x_iris.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667


In [6]:
#存取成dataFrame形式: iris['target']是類別
y_iris = pd.DataFrame(iris['target'], columns=['target'])
y_iris.head(3)

,target
0,0
1,0
2,0


In [7]:
#將資料與類別合併在一起
iris_data = pd.concat([x_iris,y_iris], axis=1)
#因為原始的SVM是一個兩類別分類的問題，因此我們在此先取兩個類別作分類
#（不過Sklearn其實也能做多類別分類的問題，所以看您分類的需求來做處理）
iris_data = iris_data[iris_data['target'].isin([0,1])]
iris_data.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,0.222222,0.625000,0.067797,0.041667,0
1,0.166667,0.416667,0.067797,0.041667,0
2,0.111111,0.500000,0.050847,0.041667,0


In [8]:
#將Iris資料隨機切分成 70%訓練資料集 和 30%測試資料集
train_X, test_X, train_y, test_y = train_test_split(
    iris_data[['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']], 
    iris_data[['target']], test_size=0.3, random_state=0)

### 三、透過Keras來做DNN分類

In [9]:
#透過Neuron 4-16-8-4-1來做DNN分類
model = Sequential()
start_unit = 16
model.add(Dense(units=start_unit, input_dim=4,kernel_initializer='uniform'))
model.add(Activation('relu'))
for i in range(0,2):
    start_unit = int(start_unit/2)
    print(start_unit)
    model.add(Dense(units=start_unit,kernel_initializer='uniform'))
    model.add(Activation('relu'))
model.add(Dense(units=1,kernel_initializer='uniform'))
model.add(Activation('sigmoid'))

8
4


In [10]:
#DNN在Keras的compile
model.compile(loss='binary_crossentropy',optimizer='adam', metrics= ['accuracy'])

In [11]:
#early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=2)

In [12]:
#DNN在Keras的運算
model.fit(x=train_X, y =train_y,epochs=50,validation_split=0.1,batch_size=8,verbose=1, callbacks=[early_stopping])

Train on 63 samples, validate on 7 samples
Epoch 1/50
63/63 [==============================] - 1s 12ms/step - loss: 0.6933 - acc: 0.3810 - val_loss: 0.6932 - val_acc: 0.4286
Epoch 2/50
63/63 [==============================] - 0s 650us/step - loss: 0.6932 - acc: 0.5079 - val_loss: 0.6932 - val_acc: 0.4286
Epoch 3/50
63/63 [==============================] - 0s 587us/step - loss: 0.6931 - acc: 0.5079 - val_loss: 0.6932 - val_acc: 0.4286
Epoch 4/50
63/63 [==============================] - 0s 476us/step - loss: 0.6931 - acc: 0.5079 - val_loss: 0.6932 - val_acc: 0.4286
Epoch 5/50
63/63 [==============================] - 0s 508us/step - loss: 0.6932 - acc: 0.5079 - val_loss: 0.6932 - val_acc: 0.4286
Epoch 6/50
63/63 [==============================] - 0s 476us/step - loss: 0.6931 - acc: 0.5079 - val_loss: 0.6933 - val_acc: 0.4286
Epoch 7/50
63/63 [==============================] - ETA: 0s - loss: 0.6934 - acc: 0.375 - 0s 460us/step - loss: 0.6931 - acc: 0.5079 - val_loss: 0.6933 - val_acc: 0.4

In [13]:
#DNN在Keras的預測結果（loss, accuracy）
scores = model.evaluate(x=test_X,y=test_y)
print(scores)

30/30 [==============================] - 0s 33us/step
[0.3196578919887543, 1.0]


In [14]:
#DNN在Keras的預測結果（y_pred）

preds = model.predict(test_X, verbose=1)
y_pred = []
for i in preds:
    if i >= 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

30/30 [==============================] - 0s 3ms/step


#### Confusion_matrix

In [15]:
from sklearn.metrics import accuracy_score, recall_score, precision_score ,confusion_matrix
accuracy = accuracy_score(test_y, y_pred)
precision = precision_score(test_y, y_pred,pos_label=1)
recall = recall_score(test_y, y_pred,pos_label=1)
tn, fp, fn, tp = confusion_matrix(test_y, y_pred).ravel()

print('Accuracy = ' + str(accuracy) +'\nPrecision = ' + str(precision) +'\nRecall = ' + str(recall))
tn, fp, fn, tp

Accuracy = 1.0
Precision = 1.0
Recall = 1.0


(15, 0, 0, 15)

In [16]:
print(classification_report(test_y, y_pred,digits=4))

             precision    recall  f1-score   support

          0     1.0000    1.0000    1.0000        15
          1     1.0000    1.0000    1.0000        15

avg / total     1.0000    1.0000    1.0000        30

